In [ ]:
%config InlineBackend.figure_formats = {"retina", "png"}
%matplotlib inline
import os
import sys

import h5py
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy import constants

sys.path.insert(0, os.path.expandvars("$HOME/GitHub/super-detector-py"))

In [ ]:
# !python "$HOME/GitHub/super-detector-py/simulate.py" "mesh.h5" "test-output.h5" -t 1e-3 -s 1e4 -e 100 -b 0.05

In [ ]:
from src.mesh.mesh import Mesh
from src.visualization.visualization_helpers import (
    get_data_range,
    load_tdgl_data,
    get_edge_observable_data,
    get_magnetic_field,
)
from src.tdgl import get_supercurrent, get_observable_on_site

In [ ]:
from typing import NamedTuple

class TDGLData(NamedTuple):
    psi: np.ndarray
    mu: np.ndarray
    a: np.ndarray
    supercurrent: np.ndarray
    normal_current: np.ndarray

In [ ]:
with h5py.File("test-output.h5", "r") as f:
    start, end = get_data_range(f)
    tdgl_data = TDGLData(*load_tdgl_data(f, end))
    mesh = Mesh.load_from_hdf5(f["mesh"])
b = get_magnetic_field("test-output.h5", end)

In [ ]:
tdgl_data

In [ ]:
directions = mesh.edge_mesh.directions
unit_vectors = directions / np.linalg.norm(directions, axis=1)[:, np.newaxis]

In [ ]:
xi = 1e-6 # meters
lambda_ = 5e-6 # meters
Phi_0 = constants.physical_constants["magn. flux quantum"][0]
mu_0 = constants.mu_0
hbar = constants.hbar
e = constants.elementary_charge
d = 0.1e-6 # meters

Lambda = lambda_**2 / d

Bc2 = Phi_0 / (2 * np.pi * xi**2) # tesla
J0 = 4 * hbar / (2 * mu_0 * e * xi * lambda_**2 / d) # A / m^2
# J0 = 4 * xi * Bc2 / (mu_0 * Lambda)

In [ ]:
J = J0 * get_observable_on_site(tdgl_data.supercurrent, mesh)

In [ ]:
# Integration path
thetas = np.linspace(0, 2 * np.pi, 501)
r = 3 * xi
x = r * np.cos(thetas)
y = r * np.sin(thetas)
coords = np.stack([x, y], axis=1)

dr = np.diff(coords, prepend=coords[:1], axis=0)

points = np.stack([mesh.x, mesh.y], axis=1) * xi

In [ ]:
J_interp = interpolate.LinearNDInterpolator(points, J)

In [ ]:
contour_supercurrent = mu_0 * Lambda * np.trapz((J_interp(coords) * dr).sum(axis=1))

In [ ]:
contour_flux = b * Bc2 * np.pi * r**2

In [ ]:
contour_supercurrent / Phi_0

In [ ]:
contour_flux / Phi_0

In [ ]:
contour_supercurrent / contour_flux

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
im = ax.tripcolor(
    points[:, 0] / xi,
    points[:, 1] / xi,
    np.linalg.norm(J, axis=1),
    cmap="inferno",
    triangles=mesh.elements,
    vmin=0,
    vmax=0.25,
)
ax.quiver(
    points[:, 0] / xi,
    points[:, 1] / xi,
    J[:, 0],
    J[:, 1],
    color="w",
)
cbar = fig.colorbar(im, ax=ax)

In [ ]:
b * Bc2

In [ ]:
import superscreen as sc

In [ ]:
ro = 5
ri = 2

layer = sc.Layer("base", london_lambda=5, thickness=0.1, z0=0)
film = sc.Polygon("film", layer=layer.name, points=sc.geometry.box(10))
hole = sc.Polygon("hole", layer=layer.name, points=sc.geometry.circle(ri))
bbox = sc.Polygon("bounding_box", layer=layer.name, points=sc.geometry.box(12))

device = sc.Device(
    "box",
    layers=[layer],
    films=[film],
#     holes=[hole],
    abstract_regions=[bbox],
)

In [ ]:
device.make_mesh(min_points=2000)

In [ ]:
device.plot(mesh=True)

In [ ]:
solution = sc.find_fluxoid_solution(
    device,
    applied_field=sc.sources.ConstantField(1e3 * b * Bc2),
    field_units="mT",
    current_units="uA",
)

In [ ]:
solution.plot_currents()

In [ ]:
f = solution.polygon_fluxoid(sc.geometry.circle(3))["base"]

In [ ]:
f

In [ ]:
f.supercurrent_part

In [ ]:
fig, ax = plt.subplots()
counts, edges, _ = ax.hist(np.linalg.norm(J, axis=1), bins=101, density=True)
_ = ax.hist(
    np.linalg.norm(solution.current_densities["base"], axis=1),
    bins=edges,
    density=True,
    alpha=0.5
)

In [ ]:
solution